In [32]:
import json
from pathlib import Path
from typing import Union
from collections import defaultdict
import hvplot.pandas
import pandas as pd
import requests
import os
from scipy.stats import linregress
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show
from citipy import citipy

from bokeh.models.tools import HoverTool
from bokeh.plotting import figure, output_file, show, save, ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Blues8

In [2]:
# see website for information:
# https://developer.nrel.gov/docs/transportation/alt-fuel-stations-v1/all/

api_key = 'q2xrwjUgqkGsBjIQRNb3V3LyeaZqV1RfKGO6Ensv'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# List of state codes
state_codes = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
               'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
               'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

# Variable to store the total number of electric vehicle stations in the US
total_stations_us = 0

# Dictionary to store the totals for each state
state_totals = {}

for state_code in state_codes:
    query_params = {
        'fuel_type_code': 'ELEC',
        'state': state_code,
        'country': 'US',
        'api_key': api_key
    }

    # Make the API request
    response = requests.get(base_url, params=query_params)

    # Parse the JSON response
    data = response.json()

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Get the total number of electric vehicle stations for the specified state
        total_stations = data.get('total_results', 0)

        # Increment the total stations in the US variable
        total_stations_us += total_stations

        # Store the result in the dictionary
        state_totals[state_code] = total_stations

    else:
        # Print an error message if the request was not successful
        print(f'Error: {response.status_code} - {response.text}')

# Sort the dictionary by state in decreasing order based on total stations
sorted_totals = dict(sorted(state_totals.items(), key=lambda item: item[1], reverse=True))

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the sorted totals to a JSON file in the 'data_query' folder
state_data_file_path = os.path.join(output_folder, 'electric_stations_by_state.json')
with open(state_data_file_path, 'w', encoding='utf-8') as state_data_file:
    json.dump(sorted_totals, state_data_file, ensure_ascii=False, indent=4)

# Print the total number of electric vehicle stations in the US
print(f'Total electric vehicle stations in the US for fuel type ELEC: {total_stations_us}')

print(f'The sorted totals have been saved to: {state_data_file_path}')


Total electric vehicle stations in the US for fuel type ELEC: 79633
The sorted totals have been saved to: data_query\electric_stations_by_state.json


In [3]:
api_key = 'AWMepE53xJxr8Qu8yP3w6J6PLAf3ye6sO7Fsdvlc'
base_url = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json'

# List of all US state codes
state_codes = [
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
    'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
    'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
]

# List of attributes to retrieve, including the new ones
attributes_to_retrieve = [
    'id', 'station_name', 'street_address', 'city', 'state', 'zip',
    'latitude', 'longitude', 'owner_type_code', 'ev_connector_types', 'ev_pricing',
    'geocode_status', 'access_code', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num', 'ev_other_evse'
]

# List to store detailed information for stations meeting the criteria
filtered_stations = []

for state_code in state_codes:
    query_params = {
        'fuel_type_code': 'ELEC',
        'state': state_code,
        'country': 'US',
        'restricted_access': 'false',  # Add restricted_access parameter
        'api_key': api_key
    }

    # Make the API request
    response = requests.get(base_url, params=query_params)

    # Parse the JSON response
    data = response.json()

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Add detailed information for each station meeting the criteria
        filtered_stations.extend([
            {attr: station[attr] for attr in attributes_to_retrieve}
            for station in data.get('fuel_stations', [])
        ])

    else:
        # Print an error message if the request was not successful
        print(f'Error: {response.status_code} - {response.text}')

# Print the total count of electric vehicle stations in the US after filtering
total_us_stations = sum(len(data.get('fuel_stations', [])) for state_code in state_codes)
print(f'Total electric vehicle stations in the US after filtering: {total_us_stations}')

# Create a folder named 'data_query' if it doesn't exist
output_folder = 'data_query'
os.makedirs(output_folder, exist_ok=True)

# Save the filtered stations to a JSON file in the 'data_query' folder
output_file_path = os.path.join(output_folder, 'filtered_stations_attributes.json')
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(filtered_stations, output_file, ensure_ascii=False, indent=4)
    
    
print(f'The filtered stations with specified attributes have been saved to: {output_file_path}')
        

Total electric vehicle stations in the US after filtering: 7100
The filtered stations with specified attributes have been saved to: data_query\filtered_stations_attributes.json


In [4]:
california_df = pd.DataFrame(filtered_stations)

In [5]:
california_df = california_df[california_df['state'] == 'CA']

In [6]:
california_df

,id,station_name,street_address,city,state,zip,latitude,longitude,owner_type_code,ev_connector_types,ev_pricing,geocode_status,access_code,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_num,ev_other_evse
2612,792,PG&E - Grass Valley Service Center,788 Taylorville Rd,Grass Valley,CA,95949,39.200620,-121.062126,T,None,None,200-9,public,NaN,NaN,NaN,None
2613,798,PG&E - Santa Cruz Service Center,615 7th Ave,Santa Cruz,CA,95062,36.969267,-121.999295,T,None,None,200-9,public,NaN,NaN,NaN,None
2614,801,PG&E - Bakersfield Service Center,4101 Wible Rd,Bakersfield,CA,93313,35.317617,-119.039048,T,None,None,200-8,public,NaN,NaN,NaN,None
2615,806,PG&E - Fresno Service Center,3754 E California Ave,Fresno,CA,93725,36.721360,-119.761165,T,None,None,200-8,public,NaN,NaN,NaN,None
2616,809,PG&E - Salinas Service Center,390 E Alisal St,Salinas,CA,93901,36.672170,-121.641106,T,None,None,200-8,public,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21916,323844,OROVILLETOYOTA OROVILLE LVL 2,1250 Oro Dam Boulevard East,Oroville,CA,95965,39.502348,-121.560024,None,[J1772],None,GPS,public,NaN,2.0,NaN,None
21917,323854,GRATON CASINO DC 1B,288 Golf Course Dr W,Rohnert Park,CA,94928,38.361705,-122.723880,None,"[CHADEMO, J1772COMBO]",None,GPS,public,NaN,NaN,1.0,None
21918,323855,WEST HERR AUTO WH FR L2-1,4545 West Ridge Road,Rochester,CA,14626,33.975073,-118.256805,None,[J1772],None,GPS,public,NaN,2.0,NaN,None
21919,323871,WOODLAND LIBRARY,501 Court St,Woodland,CA,95695,38.679106,-121.774520,None,[J1772],None,GPS,public,NaN,2.0,NaN,None


In [7]:
map_plot_3=california_df.hvplot.points(
   'longitude',
    'latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = 0.5,
    scale = 0.5,
    color = 'city',
#     hover_cols = ['owner_type_code', 'city']
)

# Display the map
map_plot_3    

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (city)

In [8]:
# Read in JSON previously created from API request
output_file_path = Path("data_query/filtered_stations_attributes.json")

# Load data from the JSON file
if output_file_path.exists():
    with output_file_path.open("r", encoding="utf8") as f:
        station_data = json.load(f)


In [9]:
# Create a DataFrame from the loaded data
station_df = pd.DataFrame(station_data)

# Organize the columns in the specified sequence
station_df = station_df[['id', 'station_name', 'street_address', 'city', 'state', 'zip',
         'latitude', 'longitude', 'owner_type_code', 'ev_connector_types', 'ev_pricing',
         'geocode_status', 'access_code', 'ev_level1_evse_num', 'ev_level2_evse_num', 'ev_dc_fast_num', 'ev_other_evse']]

# Display the DataFrame
# print(station_df.columns)

station_df.head(30)

,id,station_name,street_address,city,state,zip,latitude,longitude,owner_type_code,ev_connector_types,ev_pricing,geocode_status,access_code,ev_level1_evse_num,ev_level2_evse_num,ev_dc_fast_num,ev_other_evse
0,17,Spire - Montgomery Operations Center,2951 Chestnut St,Montgomery,AL,36107,32.367916,-86.267021,T,None,None,200-9,private,NaN,NaN,NaN,None
1,597,Spire,2828 Dauphin St,Mobile,AL,36606,30.689832,-88.108861,T,None,None,200-9,private,NaN,NaN,NaN,None
2,13712,Gala Gas Co Inc,54 Old US Highway 82,Eufaula,AL,36027,31.912022,-85.150318,P,None,None,GPS,private,NaN,NaN,NaN,None
3,13723,Superior Gas,421 Noble St,Anniston,AL,36201,33.642079,-85.828466,P,None,None,GPS,public,NaN,NaN,NaN,None
4,13725,Superior Gas,702 Memorial Dr,Piedmont,AL,36272,33.921589,-85.620314,P,None,None,200-9,public,NaN,NaN,NaN,None
5,13726,Superior Gas,915 Lafayette Hwy,Roanoke,AL,36274,33.136025,-85.362562,P,None,None,GPS,public,NaN,NaN,NaN,None
6,13727,Blossman Gas Inc,1662 Old Highway 431,Wedowee,AL,36278,33.331528,-85.476368,P,None,None,GPS,public,NaN,NaN,NaN,None
7,13748,Blossman Gas Inc,3003 Marvyn Pkwy,Opelika,AL,36804,32.613832,-85.371078,P,None,None,200-9,public,NaN,NaN,NaN,None
8,14970,Blossman Gas Inc,305 N Schillinger,Mobile,AL,36608,30.697377,-88.226620,P,None,None,200-9,public,NaN,NaN,NaN,None
9,17480,Allgas,3600 Meridian St N,Huntsville,AL,35811,34.773256,-86.571778,P,None,None,200-9,public,NaN,NaN,NaN,None


In [10]:
# Export to csv

station_df.to_csv("data_query/stations.csv", index_label="Index")

In [11]:
# new data frame with fewer columns
station_spread_df = pd.DataFrame(station_data)

station_spread_df = station_spread_df[['id', 'station_name', 'street_address', 'city', 'state', 'zip',
         'latitude', 'longitude', 'owner_type_code']]
 
station_spread_df.head()

,id,station_name,street_address,city,state,zip,latitude,longitude,owner_type_code
0,17,Spire - Montgomery Operations Center,2951 Chestnut St,Montgomery,AL,36107,32.367916,-86.267021,T
1,597,Spire,2828 Dauphin St,Mobile,AL,36606,30.689832,-88.108861,T
2,13712,Gala Gas Co Inc,54 Old US Highway 82,Eufaula,AL,36027,31.912022,-85.150318,P
3,13723,Superior Gas,421 Noble St,Anniston,AL,36201,33.642079,-85.828466,P
4,13725,Superior Gas,702 Memorial Dr,Piedmont,AL,36272,33.921589,-85.620314,P


In [12]:
station_df.dtypes

id                      int64
station_name           object
street_address         object
city                   object
state                  object
zip                    object
latitude              float64
longitude             float64
owner_type_code        object
ev_connector_types     object
ev_pricing             object
geocode_status         object
access_code            object
ev_level1_evse_num    float64
ev_level2_evse_num    float64
ev_dc_fast_num        float64
ev_other_evse          object
dtype: object

In [13]:
station_df.count()


id                    79633
station_name          79633
street_address        79600
city                  79633
state                 79633
zip                   79633
latitude              79633
longitude             79633
owner_type_code       29658
ev_connector_types    68298
ev_pricing            13666
geocode_status        79629
access_code           79632
ev_level1_evse_num      712
ev_level2_evse_num    59610
ev_dc_fast_num         9614
ev_other_evse            40
dtype: int64

In [14]:
filepath = Path("data_query/filtered_stations_attributes.json")
with open(filepath, encoding="utf-8") as jsonfile:
    filtered_stations_json = json.load(jsonfile)


In [15]:
type(filtered_stations_json)

list

In [16]:
filtered_stations_json

[{'id': 17,
  'station_name': 'Spire - Montgomery Operations Center',
  'street_address': '2951 Chestnut St',
  'city': 'Montgomery',
  'state': 'AL',
  'zip': '36107',
  'latitude': 32.367916,
  'longitude': -86.267021,
  'owner_type_code': 'T',
  'ev_connector_types': None,
  'ev_pricing': None,
  'geocode_status': '200-9',
  'access_code': 'private',
  'ev_level1_evse_num': None,
  'ev_level2_evse_num': None,
  'ev_dc_fast_num': None,
  'ev_other_evse': None},
 {'id': 597,
  'station_name': 'Spire',
  'street_address': '2828 Dauphin St',
  'city': 'Mobile',
  'state': 'AL',
  'zip': '36606',
  'latitude': 30.689832,
  'longitude': -88.108861,
  'owner_type_code': 'T',
  'ev_connector_types': None,
  'ev_pricing': None,
  'geocode_status': '200-9',
  'access_code': 'private',
  'ev_level1_evse_num': None,
  'ev_level2_evse_num': None,
  'ev_dc_fast_num': None,
  'ev_other_evse': None},
 {'id': 13712,
  'station_name': 'Gala Gas Co Inc',
  'street_address': '54 Old US Highway 82',
  '

In [17]:
filepath = Path("data_query/electric_stations_by_state.json")
with open(filepath, encoding="utf-8") as jsonfile:
    electric_stations_by_state_json = json.load(jsonfile)

In [18]:
type(electric_stations_by_state_json)

dict

In [19]:
electric_stations_by_state_json

{'CA': 19309,
 'NY': 4374,
 'TX': 4163,
 'FL': 4046,
 'MA': 3200,
 'WA': 2550,
 'CO': 2512,
 'GA': 2291,
 'IL': 2289,
 'PA': 2186,
 'OH': 2036,
 'NC': 2021,
 'MD': 2008,
 'MI': 1916,
 'VA': 1868,
 'MN': 1563,
 'MO': 1546,
 'AZ': 1465,
 'NJ': 1441,
 'OR': 1436,
 'TN': 1140,
 'IA': 1099,
 'UT': 1022,
 'WI': 1012,
 'IN': 947,
 'CT': 931,
 'SC': 749,
 'KS': 723,
 'OK': 707,
 'NV': 686,
 'AL': 581,
 'ME': 511,
 'AR': 500,
 'KY': 441,
 'HI': 434,
 'NE': 432,
 'VT': 414,
 'NM': 385,
 'LA': 371,
 'RI': 351,
 'NH': 295,
 'MS': 255,
 'DE': 248,
 'ID': 247,
 'WV': 214,
 'SD': 198,
 'MT': 158,
 'ND': 154,
 'WY': 142,
 'AK': 66}

In [20]:
len(electric_stations_by_state_json)

50

In [21]:
stations_by_state_df = pd.DataFrame(electric_stations_by_state_json, index=[0])
stations_by_state_df.head()

,CA,NY,TX,FL,MA,WA,CO,GA,IL,PA,...,NH,MS,DE,ID,WV,SD,MT,ND,WY,AK
0,19309,4374,4163,4046,3200,2550,2512,2291,2289,2186,...,295,255,248,247,214,198,158,154,142,66


In [22]:
List = list(electric_stations_by_state_json.items())

In [28]:
stations_by_state_df1 = pd.DataFrame(List, columns=["State", "EV Stations"])

stations_by_state_df1

,State,EV Stations
0,CA,19309
1,NY,4374
2,TX,4163
3,FL,4046
4,MA,3200
5,WA,2550
6,CO,2512
7,GA,2291
8,IL,2289
9,PA,2186


In [29]:
stations_by_state_df1.dtypes

State          object
EV Stations     int64
dtype: object

In [30]:
stations_by_state_df1['State'] = stations_by_state_df1['State'].astype("string")

In [31]:
stations_by_state_df1.dtypes

State          string[python]
EV Stations             int64
dtype: object

In [24]:
type(stations_by_state_df2)

pandas.core.frame.DataFrame

In [25]:
stations_by_state_df2 = stations_by_state_df2.set_index("State")
stations_by_state_df2

,EV Stations
State,
CA,19309
NY,4374
TX,4163
FL,4046
MA,3200
WA,2550
CO,2512
GA,2291
IL,2289


In [27]:
stations_by_state_df2.dtypes

EV Stations    int64
dtype: object

In [26]:
State = stations_by_state_df2['State']
EV_Stations = stations_by_state_df2['EV Stations']

output_file('index.html')

#  add plot
chart = figure(
        y_range=State,
        width=800,
        height=600,
        title='EV Stations by State',
        x_axis_label='EV Stations',
        tools="pan,box_select,zoom_in,zoom_out,save"
) 

# Rende glyph
chart.hbar(y=State,
           right=EV_Stations,
           left=0,
           height=0.4,
           color='blue',
           fill_alpha=0.5
)

# show results
show(chart)


KeyError: 'State'

In [ ]:
# loading csv
alt_fuel_stations_data_to_load = Path("data_query/alt_fuel_stations (Dec 31 2023).csv")

# read csv
fuel_station_data_df = pd.read_csv(alt_fuel_stations_data_to_load)
fuel_station_data_df


In [ ]:
fuel_station_data_df.columns

In [ ]:
fuel_station_data_df = fuel_station_data_df[['Fuel Type Code', 'Station Name', 'City', 'State', 'ZIP', 
                                             'Status Code', 'Latitude', 'Longitude', 'EV Connector Types'
    
]]
fuel_station_data_df.head()

In [ ]:
geoapify_key = "cf8398abc3a34e5696e15483b6361d24"